In [1]:
# We found that certain columns are a sum of the columns that follow it, such as Power is a sum of Shot Power, Jumping,
# Stamina, Strength and Long Shots. Before we Train-Test, we can try drop the columns that sum up to the sumed column
# and also try dropping the sumed column to see which gives us a better output
# The '+' in the position columns indicate the potential improvement score in that position
# IR = International Reputation can be dropped
# Keeping ID to be able to search indivuals for now, but dropping when we come to train the model
# Could drop BP as we see that the first listed Position is the players best position
# We can drop Team & Contract as Team is a reflection of Club and we have a Contract column
# Convert Loan Date End to a Boolan as we have the end date in the contract
# For Joined, we will only keep the year as the exact date doesn't give us much value
# Converting Height to cm and excluding the metric name, as well as for Weight to kg
# Converting Value to actual numeric numnber e.g 6M to 6,000,000. Same with Wage and Release Clause
# Altering the Contract for the players on loan to reflect the start year(coming from Joined) and dropping the 'Loan' string
# so that all values in Contract will have a start year and an end year. Once this has been done, we can drop Joined column 
# https://fifauteam.com/fifa-21-attributes-guide/ useful to understanding the attributes
# Hits column is unknown for now, but it seems to have some correlation to the overal score.
# Position columns have the current value with '+' for their potential growth. We will split this column into 2, original column
# to reflect the current value and the new column will be the sum of the original column plus the potential value
# Keeping OVA as that is our Label

In [2]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import functools
from datetime import date
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn import linear_model

In [3]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [4]:
fifa = pd.read_csv('./fifa21_train.csv')
fifa.columns = [i.lower().replace(' ', '_') for i in fifa.columns]

In [5]:
fifa.drop(columns = ['position','team_&_contract','loan_date_end', 'contract', 'joined'], axis = 1, inplace = True)

In [6]:
lst = fifa.columns.unique()
fifa.dtypes

id                    int64
name                 object
age                   int64
nationality          object
club                 object
bp                   object
height               object
weight               object
foot                 object
growth                int64
value                object
wage                 object
release_clause       object
attacking             int64
crossing              int64
finishing             int64
heading_accuracy      int64
short_passing         int64
volleys             float64
skill                 int64
dribbling             int64
curve               float64
fk_accuracy           int64
long_passing          int64
ball_control          int64
movement              int64
acceleration          int64
sprint_speed          int64
agility             float64
reactions             int64
balance             float64
power                 int64
shot_power            int64
jumping             float64
stamina               int64
strength            

In [7]:
def minus(df):
    lst = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']
    for i in lst:
        df[[i, i + '_pot']] = df[i].str.split('+', expand=True)
        df[i + '_pot'] = df[i + '_pot'].astype('int') + df[i].astype('int')
        df[i] = df[i].astype('int')
    return df

In [8]:
minus(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,Right,1,€525K,€4K,€801K,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,Right,0,€8.5M,€23K,€0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,Right,0,€9M,€49K,€15.3M,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,80,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,Right,13,€275K,€4K,€694K,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,59,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,Right,8,€725K,€2K,€1.4M,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,65,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,"6'1""",168lbs,Left,5,€625K,€3K,€891K,224,62,24,51,59,28.0,248,57,49.0,29,54,59,333,78,73,68.0,53,61.0,273,31,72.0,77,68,25,266,63,59.0,51.0,54.0,39,46.0,184,59,64,61.0,61,13,14,14,6,14,1589,347,3 ★,2★,Medium,Medium,1 ★,75,28,56,59,60,69,3,48,48,48,55,52,52,52,55,54,54,54,58,56,

In [9]:
fifa.isna().sum()

id                    0
name                  0
age                   0
nationality           0
club                 18
bp                    0
height                0
weight                0
foot                  0
growth                0
value                 0
wage                  0
release_clause        0
attacking             0
crossing              0
finishing             0
heading_accuracy      0
short_passing         0
volleys              41
skill                 0
dribbling             0
curve                41
fk_accuracy           0
long_passing          0
ball_control          0
movement              0
acceleration          0
sprint_speed          0
agility              41
reactions             0
balance              41
power                 0
shot_power            0
jumping              41
stamina               0
strength              0
long_shots            0
mentality             0
aggression            0
interceptions         7
positioning           7
vision          

In [10]:
fifa.dropna(inplace=True) #We guess the NaN values are for players who are very young and don't have proper ratings yet, so we are dropping them.

In [11]:
fifa.drop_duplicates(inplace = True)

In [12]:
 def star(df):
    
    lst_str = ['w/f', 'sm', 'ir']
    
    for i in lst_str:
            
        df[[i]] = df[[i]].replace('★', '', regex=True)
        df[[i]] = df[[i]].astype('int')
        
    return df

In [13]:
star(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,Right,1,€525K,€4K,€801K,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,High,Medium,1,69,51,63,63,51,60,3,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,Right,0,€8.5M,€23K,€0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,High,Low,2,83,75,68,82,33,71,44,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,Right,0,€9M,€49K,€15.3M,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,High,Medium,2,80,77,78,86,27,56,73,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,80,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,Right,13,€275K,€4K,€694K,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,Medium,Medium,1,57,44,54,57,57,60,7,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,59,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,Right,8,€725K,€2K,€1.4M,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,Low,Medium,1,66,44,60,64,60,66,4,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,65,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,"5'7""",154lbs,Right,0,€550K,€500,€1.2M,290,36,72,65,57,60.0,283,62,56.0,56,46,63,368,67,65,76.0,71,89.0,303,64,58.0,56,60,65,254,50,33.0,64.0,37.0,70,60.0,80,41,19,20.0,60,12,13,8,13,14,1638,337,3,3,High,Low,1,66,68,47,66,33,57,4,65,65,65,61,63,63,63,61,60,60,60,58,53,53,53,58,46,45,45,45,46,44,43,43,43,44,19,65,65,65,65

In [14]:
fifa.dtypes

id                    int64
name                 object
age                   int64
nationality          object
club                 object
bp                   object
height               object
weight               object
foot                 object
growth                int64
value                object
wage                 object
release_clause       object
attacking             int64
crossing              int64
finishing             int64
heading_accuracy      int64
short_passing         int64
volleys             float64
skill                 int64
dribbling             int64
curve               float64
fk_accuracy           int64
long_passing          int64
ball_control          int64
movement              int64
acceleration          int64
sprint_speed          int64
agility             float64
reactions             int64
balance             float64
power                 int64
shot_power            int64
jumping             float64
stamina               int64
strength            

In [15]:
 def money(df):
    
    lst_mny = ['value', 'wage', 'release_clause', 'hits']
    
    for i in lst_mny:
            
        
        df[i] = df[i].replace({'€': '', 'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(float)
        
    return df

In [16]:
money(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",161lbs,Right,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,High,Medium,1,69,51,63,63,51,60,3.0,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",159lbs,Right,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,High,Low,2,83,75,68,82,33,71,44.0,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",134lbs,Right,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,High,Medium,2,80,77,78,86,27,56,73.0,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,80,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,"5'10""",152lbs,Right,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,Medium,Medium,1,57,44,54,57,57,60,7.0,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,59,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",150lbs,Right,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,Low,Medium,1,66,44,60,64,60,66,4.0,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,65,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,"5'7""",154lbs,Right,0,550000.0,500.0,1200000.0,290,36,72,65,57,60.0,283,62,56.0,56,46,63,368,67,65,76.0,71,89.0,303,64,58.0,56,60,65,254,50,33.0,64.0,37.0,70,60.0,80,41,19,20.0,60,12,13,8,13,14,1638,337,3,3,High,Low,1,66,68,47,66,33,57,4.0,65,65,65,61,63,63,63,61

In [17]:
def weight(df):
    
    lst_weight = ['weight']
    for i in lst_weight:
        df[i] = df[i].replace({'lbs':''}, regex=True).astype(int)
        df[i] = round(df[i]/2.205, 2) 
    
    return df

In [18]:
weight(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,"5'9""",73.02,Right,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,High,Medium,1,69,51,63,63,51,60,3.0,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,"6'0""",72.11,Right,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,High,Low,2,83,75,68,82,33,71,44.0,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,"5'4""",60.77,Right,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,High,Medium,2,80,77,78,86,27,56,73.0,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,80,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,"5'10""",68.93,Right,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,Medium,Medium,1,57,44,54,57,57,60,7.0,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,59,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,"5'11""",68.03,Right,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,Low,Medium,1,66,44,60,64,60,66,4.0,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,65,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,"5'7""",69.84,Right,0,550000.0,500.0,1200000.0,290,36,72,65,57,60.0,283,62,56.0,56,46,63,368,67,65,76.0,71,89.0,303,64,58.0,56,60,65,254,50,33.0,64.0,37.0,70,60.0,80,41,19,20.0,60,12,13,8,13,14,1638,337,3,3,High,Low,1,66,68,47,66,33,57,4.0,65,65,65,61,63,63,63,61,60,60

In [19]:
def height(df):
    lst_height =['height']
    for i in lst_height:
        df[i] = df[i].replace({'"':''}, regex=True)
        df[i] = df[i].apply(lambda x: 12*int(x.split("'")[0])+int(x.split("'")[1]))
        df[i] = df[i]*2.54
    return df

In [20]:
height(fifa)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,175.26,73.02,Right,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,High,Medium,1,69,51,63,63,51,60,3.0,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,182.88,72.11,Right,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,High,Low,2,83,75,68,82,33,71,44.0,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,162.56,60.77,Right,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,High,Medium,2,80,77,78,86,27,56,73.0,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,80,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,177.80,68.93,Right,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,Medium,Medium,1,57,44,54,57,57,60,7.0,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,59,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,180.34,68.03,Right,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,Low,Medium,1,66,44,60,64,60,66,4.0,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,65,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11695,196848,I. Ávila,29,Peru,Melgar FBC,ST,170.18,69.84,Right,0,550000.0,500.0,1200000.0,290,36,72,65,57,60.0,283,62,56.0,56,46,63,368,67,65,76.0,71,89.0,303,64,58.0,56,60,65,254,50,33.0,64.0,37.0,70,60.0,80,41,19,20.0,60,12,13,8,13,14,1638,337,3,3,High,Low,1,66,68,47,66,33,57,4.0,65,65,65,61,63,63,63,61,60,60,60,58,5

In [21]:
fifa.head()

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,175.26,73.02,Right,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,High,Medium,1,69,51,63,63,51,60,3.0,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,182.88,72.11,Right,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,High,Low,2,83,75,68,82,33,71,44.0,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,162.56,60.77,Right,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,High,Medium,2,80,77,78,86,27,56,73.0,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,80,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14
3,233796,J. Evans,22,Wales,Swansea City,CDM,177.80,68.93,Right,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,Medium,Medium,1,57,44,54,57,57,60,7.0,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,59,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,180.34,68.03,Right,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,Low,Medium,1,66,44,60,64,60,66,4.0,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,65,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17


In [22]:
fifa_cat = fifa.select_dtypes('object').drop(['name', 'nationality', 'club'], axis = 1) #We decide to drop the nationality, club and name since we don't think the have much value

In [23]:
fifa_num = fifa._get_numeric_data().drop(['id'], axis = 1) #We also drop the 'id' since it's unique for every player and would just create noise in un model.

In [24]:
X_num = fifa_num.drop(['ova'], axis = 1)

In [25]:
y = fifa_num['ova']

In [26]:
X_num.head()

,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,26,175.26,73.02,1,525000.0,4000.0,801000.0,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,1,69,51,63,63,51,60,3.0,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,59,59,59,61,62,62,62,61,64,64,64,64,64,64,64,64,60,60,60,60,60,59,55,55,55,59,16
1,30,182.88,72.11,0,8500000.0,23000.0,0.0,365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,2,83,75,68,82,33,71,44.0,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77,77,77,77,77,77,77,77,77,77,77,77,70,70,70,77,59,55,55,55,59,55,50,50,50,55,20
2,33,162.56,60.77,0,9000000.0,49000.0,15300000.0,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,2,80,77,78,86,27,56,73.0,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,75,75,75,80,79,79,79,80,80,80,80,80,76,76,76,80,61,58,58,58,61,55,43,43,43,55,14
3,22,177.80,68.93,13,275000.0,4000.0,694000.0,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,1,57,44,54,57,57,60,7.0,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,52,52,52,51,51,51,51,51,55,55,55,55,58,58,58,55,58,60,60,60,58,59,60,60,60,59,16
4,23,180.34,68.03,8,725000.0,2000.0,1400000.0,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,1,66,44,60,64,60,66,4.0,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,58,58,58,59,59,59,59,59,63,63,63,64,65,65,65,64,66,66,66,66,66,65,63,63,63,65,17


In [27]:
transformer = MinMaxScaler().fit(X_num)
X_num_n = transformer.transform(X_num)
X_num_n = pd.DataFrame(X_num_n, columns=X_num.columns)
X_num_n.head()

,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,0.370370,0.40,0.349317,0.038462,0.005833,0.007143,0.004811,0.546835,0.545455,0.478261,0.431818,0.710843,0.465116,0.569087,0.615385,0.444444,0.561798,0.642857,0.637363,0.663818,0.607143,0.729412,0.573171,0.591549,0.8125,0.556270,0.602410,0.685714,0.694118,0.454545,0.455556,0.654596,0.517241,0.552941,0.645161,0.682353,0.547619,0.500000,0.518219,0.516854,0.595238,0.440476,0.084309,0.056818,0.116279,0.139535,0.078652,0.045455,0.604577,0.492366,0.75,0.25,0.00,0.614286,0.447368,0.558824,0.522388,0.506494,0.515625,0.000000,0.565789,0.565789,0.565789,0.602564,0.602564,0.602564,0.602564,0.602564,0.610390,0.610390,0.610390,0.631579,0.643836,0.643836,0.643836,0.631579,0.642857,0.614286,0.614286,0.614286,0.642857,0.637681,0.527778,0.527778,0.527778,0.637681,0.075949,0.565789,0.565789,0.565789,0.602564,0.602564,0.602564,0.602564,0.602564,0.613333,0.613333,0.613333,0.618421,0.630137,0.630137,0.630137,0.618421,0.619718,0.591549,0.591549,0.591549,0.619718,0.619718,0.521127,0.521127,0.521127,0.619718,0.0625
1,0.518519,0.55,0.333392,0.000000,0.094444,0.041071,0.000000,0.817722,0.681818,0.826087,0.806818,0.686747,0.837209,0.777518,0.857143,0.822222,0.752809,0.642857,0.813187,0.829060,0.833333,0.847059,0.902439,0.718310,0.7250,0.768489,0.746988,0.800000,0.741176,0.701299,0.711111,0.718663,0.517241,0.329412,0.817204,0.729412,0.809524,0.690476,0.230769,0.359551,0.166667,0.190476,0.100703,0.102273,0.058140,0.139535,0.056180,0.159091,0.781945,0.702290,0.50,0.75,0.25,0.814286,0.763158,0.632353,0.805970,0.272727,0.687500,0.004458,0.815789,0.815789,0.815789,0.807692,0.794872,0.794872,0.794872,0.807692,0.779221,0.779221,0.779221,0.802632,0.712329,0.712329,0.712329,0.802632,0.614286,0.528571,0.528571,0.528571,0.614286,0.565217,0.444444,0.444444,0.444444,0.565217,0.113924,0.802632,0.802632,0.802632,0.807692,0.794872,0.794872,0.794872,0.807692,0.786667,0.786667,0.786667,0.789474,0.712329,0.712329,0.712329,0.789474,0.605634,0.521127,0.521127,0.521127,0.605634,0.563380,0.450704,0.450704,0.450704,0.563380,0.1125
2,0.629630,0.15,0.134932,0.000000,0.100000,0.087500,0.091892,0.744304,0.761364,0.793478,0.329545,0.807229,0.825581,0.892272,0.879121,0.944444,0.966292,0.773810,0.879121,0.886040,0.845238,0.764706,0.963415,0.760563,0.9500,0.562701,0.807229,0.128571,0.741176,0.285714,0.822222,0.771588,0.758621,0.247059,0.838710,0.800000,0.773810,0.833333,0.242915,0.224719,0.273810,0.261905,0.021077,0.045455,0.011628,0.046512,0.011236,0.011364,0.759059,0.671756,0.75,0.75,0.25,0.771429,0.789474,0.779412,0.865672,0.194805,0.453125,0.007611,0.763158,0.763158,0.763158,0.846154,0.820513,0.820513,0.820513,0.846154,0.831169,0.831169,0.831169,0.842105,0.794521,0.794521,0.794521,0.842105,0.642857,0.571429,0.571429,0.571429,0.642857,0.565217,0.347222,0.347222,0.347222,0.565217,0.037975,0.776316,0.776316,0.776316,0.846154,0.820513,0.820513,0.820513,0.846154,0.826667,0.826667,0.826667,0.828947,0.794521,0.794521,0.794521,0.828947,0.633803,0.563380,0.563380,0.563380,0.633803,0.563380,0.352113,0.352113,0.352113,0.563380,0.0375
3,0.222222,0.45,0.277739,0.500000,0.003056,0.007143,0.004168,0.506329,0.431818,0.423913,0.602273,0.614458,0.372093,0.505855,0.53846

In [28]:
fifa_num_corr = fifa_num.corr()
fifa_num_corr.round(2)

,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
age,1.00,0.09,0.25,-0.87,0.11,0.20,0.09,0.19,0.16,0.11,0.17,0.18,0.18,0.17,0.04,0.18,0.22,0.23,0.12,-0.04,-0.17,-0.17,-0.03,0.50,-0.08,0.33,0.31,0.20,0.09,0.35,0.20,0.28,0.28,0.19,0.12,0.24,0.19,0.43,0.12,0.17,0.11,0.09,0.12,0.12,0.12,0.12,0.13,0.12,0.29,0.42,0.09,0.08,0.37,-0.16,0.31,0.40,0.24,0.23,0.44,-0.08,0.18,0.18,0.18,0.12,0.15,0.15,0.15,0.12,0.15,0.15,0.15,0.13,0.21,0.21,0.21,0.13,0.16,0.23,0.23,0.23,0.16,0.16,0.21,0.21,0.21,0.16,0.15,0.52,0.17,0.17,0.17,0.11,0.15,0.15,0.15,0.11,0.14,0.14,0.14,0.12,0.21,0.21,0.21,0.12,0.16,0.22,0.22,0.22,0.16,0.16,0.20,0.20,0.20,0.16,0.15
height,0.09,1.00,0.77,-0.05,-0.00,0.02,-0.00,-0.35,-0.49,-0.36,0.06,-0.35,-0.34,-0.46,-0.48,-0.44,-0.40,-0.32,-0.41,-0.64,-0.55,-0.46,-0.64,-0.00,-0.78,-0.13,-0.17,0.00,-0.28,0.56,-0.38,-0.31,-0.02,-0.04,-0.43,-0.37,-0.31,-0.15,-0.06,-0.05,-0.05,-0.06,0.35,0.34,0.35,0.34,0.35,0.35,-0.37,-0.12,-0.16,-0.43,0.05,-0.38,-0.09,-0.21,-0.31,0.09,0.44,-0.01,-0.35,-0.35,-0.35,-0.48,-0.44,-0.44,-0.44,-0.48,-0.46,-0.46,-0.46,-0.48,-0.39,-0.39,-0.39,-0.48,-0.30,-0.18,-0.18,-0.18,-0.30,-0.24,-0.03,-0.03,-0.03,-0.24,0.35,0.02,-0.36,-0.36,-0.36,-0.48,-0.44,-0.44,-0.44,-0.48,-0.45,-0.45,-0.45,-0.47,-0.39,-0.39,-0.39,-0.47,-0.30,-0.18,-0.18,-0.18,-0.30,-0.24,-0.04,-0.04,-0.04,-0.24,0.35
weight,0.25,0.77,1.00,-0.22,0.04,0.07,0.03,-0.25,-0.39,-0.27,0.10,-0.26,-0.23,-0.35,-0.40,-0.34,-0.30,-0.24,-0.32,-0.54,-0.49,-0.41,-0.56,0.11,-0.66,0.01,-0.04,0.08,-0.21,0.65,-0.26,-0.20,0.06,-0.02,-0.33,-0.27,-0.21,-0.04,-0.04,-0.03,-0.04,-0.05,0.32,0.32,0.32,0.31,0.32,0.32,-0.25,0.00,-0.12,-0.35,0.11,-0.34,0.01,-0.10,-0.21,0.10,0.54,-0.03,-0.24,-0.24,-0.24,-0.38,-0.33,-0.33,-0.33,-0.38,-0.35,-0.35,-0.35,-0.37,-0.29,-0.29,-0.29,-0.37,-0.23,-0.11,-0.11,-0.11,-0.23,-0.18,0.02,0.02,0.02,-0.18,0.33,0.15,-0.25,-0.25,-0.25,-0.38,-0.33,-0.33,-0.33,-0.38,-0.35,-0.35,-0.35,-0.37,-0.29,-0.29,-0.29,-0.37,-0.23,-0.12,-0.12,-0.12,-0.23,-0.18,0.01,0.01,0.01,-0.18,0.33
growth,-0.87,-0.05,-0.22,1.00,-0.19,-0.23,-0.17,-0.28,-0.25,-0.19,-0.23,-0.27,-0.25,-0.26,-0.16,-0.25,-0.27,-0.30,-0.23,-0.10,0.02,0.01,-0.08,-0.56,0.01,-0.43,-0.37,-0.23,-0.24,-0.37,-0.28,-0.35,-0.32,-0.22,-0.21,-0.30,-0.23,-0.47,-0.16,-0.20,-0.15,-0.12,-0.05,-0.05,-0.05,-0.05,-0.06,-0.05,-0.40,-0.52,-0.11,-0.17,-0.27,0.01,-0.34,-0.44,-0.32,-0.24,-0.49,0.07,-0.28,-0.28,-0.28,-0.23,-0.26,-0.26,-0.26,-0.23,-0.26,-0.26,-0.26,-0.25,-0.31,-0.31,-0.31,-0.25,-0.26,-0.30,-0.30,-0.30,-0.26,-0.25,-0.27,-0.27,-0.27,-0.25,-0.09,-0.60,-0.28,-0.28,-0.28,-0.23,-0.26,-0.26,-0.26,-0.23,-0.25,-0.25,-0.25,-0.24,-0.31,-0.31,-0.31,-0.24,-0.26,-0.30,-0.30,-0.30,-0.26,-0.24,-0.26,-0.26,-0.26,-0.24,-0.09
value,0.11,-0.00,0.04,-0.19,1.00,0.83,0.98,0.29,0.25,0.23,0.17,0.33,0.26,0.30,0.26,0.28,0.24,0.31,0.29,0.26,0.16,0.17,0.19,0.57,0.12,0.33,0.34,0.13,0.23,0.13,0.26,0.31,0.19,0.15,0.25,0.36,0.21,0.44,0.13,0.15,0.12,0.10,0.01,0.01,0.01,0.01,0.01,0.01,0.39,0.53,0.15,0.28,0.51,0.22,0.33,0.45,0.45,0.17,0.27,0.38,0.32,0.32,0.32,0.31,0.32,0.32,0.32,0.31,0.33,0.33,0.33,0.32,0.35,0.35,0.35,0.32,0.27,0.27,0.27,0.27,0.27,0.25,0.20,0.20,0.20,0.25,0.05,0.64,0.32,0.32,0.32,0.31,0.33,0.33,0.33,0.31,0.34,0.34,0.34,0.33,0.36,0.36,0.36,0.33,0.

In [29]:
fifa_cat.head()

,bp,foot,a/w,d/w
0,CM,Right,High,Medium
1,ST,Right,High,Low
2,CAM,Right,High,Medium
3,CDM,Right,Medium,Medium
4,CDM,Right,Low,Medium


In [30]:
fifa_cat_hot = fifa_cat[['bp', 'foot']] #We decide to OneHot encode these two variables because they are categorical with no logical value order.
fifa_cat_hot.head()

,bp,foot
0,CM,Right
1,ST,Right
2,CAM,Right
3,CDM,Right
4,CDM,Right


In [31]:
fifa_cat_ord = fifa_cat[['a/w', 'd/w']] #We decide to Ordinal encode these variables because there is a logical order to the values they can take, and we want to give that some weight.
fifa_cat_ord.head()

,a/w,d/w
0,High,Medium
1,High,Low
2,High,Medium
3,Medium,Medium
4,Low,Medium


In [32]:
hot_encoder = OneHotEncoder(drop='first').fit(fifa_cat_hot)
hot_encoded = hot_encoder.transform(fifa_cat_hot).toarray()
hot_cols = hot_encoder.get_feature_names(input_features=fifa_cat_hot.columns)
hot_encoded = pd.DataFrame(hot_encoded, columns = [i.lower() for i in hot_cols])
hot_encoded.head()

,bp_cb,bp_cdm,bp_cf,bp_cm,bp_gk,bp_lb,bp_lm,bp_lw,bp_lwb,bp_rb,bp_rm,bp_rw,bp_rwb,bp_st,foot_right
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [33]:
ord_encoder = OrdinalEncoder(categories = [['Low', 'Medium', 'High'],['Low', 'Medium', 'High']]).fit(fifa_cat_ord)

ord_encoded = pd.DataFrame((ord_encoder.transform(fifa_cat_ord)), 
                           columns = fifa_cat_ord.columns)
ord_encoded.head()

,a/w,d/w
0,2.0,1.0
1,2.0,0.0
2,2.0,1.0
3,1.0,1.0
4,0.0,1.0


In [34]:
transformer_ord = MinMaxScaler().fit(ord_encoded)
ord_encoded_n = transformer_ord.transform(ord_encoded)
ord_encoded_n = pd.DataFrame(ord_encoded_n, columns=fifa_cat_ord.columns)
ord_encoded_n.head()

,a/w,d/w
0,1.0,0.5
1,1.0,0.0
2,1.0,0.5
3,0.5,0.5
4,0.0,0.5


In [35]:
X = pd.concat([ord_encoded_n, hot_encoded, X_num_n],axis=1)
X.head()

,a/w,d/w,bp_cb,bp_cdm,bp_cf,bp_cm,bp_gk,bp_lb,bp_lm,bp_lw,bp_lwb,bp_rb,bp_rm,bp_rw,bp_rwb,bp_st,foot_right,age,height,weight,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,1.0,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.370370,0.40,0.349317,0.038462,0.005833,0.007143,0.004811,0.546835,0.545455,0.478261,0.431818,0.710843,0.465116,0.569087,0.615385,0.444444,0.561798,0.642857,0.637363,0.663818,0.607143,0.729412,0.573171,0.591549,0.8125,0.556270,0.602410,0.685714,0.694118,0.454545,0.455556,0.654596,0.517241,0.552941,0.645161,0.682353,0.547619,0.500000,0.518219,0.516854,0.595238,0.440476,0.084309,0.056818,0.116279,0.139535,0.078652,0.045455,0.604577,0.492366,0.75,0.25,0.00,0.614286,0.447368,0.558824,0.522388,0.506494,0.515625,0.000000,0.565789,0.565789,0.565789,0.602564,0.602564,0.602564,0.602564,0.602564,0.610390,0.610390,0.610390,0.631579,0.643836,0.643836,0.643836,0.631579,0.642857,0.614286,0.614286,0.614286,0.642857,0.637681,0.527778,0.527778,0.527778,0.637681,0.075949,0.565789,0.565789,0.565789,0.602564,0.602564,0.602564,0.602564,0.602564,0.613333,0.613333,0.613333,0.618421,0.630137,0.630137,0.630137,0.618421,0.619718,0.591549,0.591549,0.591549,0.619718,0.619718,0.521127,0.521127,0.521127,0.619718,0.0625
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.518519,0.55,0.333392,0.000000,0.094444,0.041071,0.000000,0.817722,0.681818,0.826087,0.806818,0.686747,0.837209,0.777518,0.857143,0.822222,0.752809,0.642857,0.813187,0.829060,0.833333,0.847059,0.902439,0.718310,0.7250,0.768489,0.746988,0.800000,0.741176,0.701299,0.711111,0.718663,0.517241,0.329412,0.817204,0.729412,0.809524,0.690476,0.230769,0.359551,0.166667,0.190476,0.100703,0.102273,0.058140,0.139535,0.056180,0.159091,0.781945,0.702290,0.50,0.75,0.25,0.814286,0.763158,0.632353,0.805970,0.272727,0.687500,0.004458,0.815789,0.815789,0.815789,0.807692,0.794872,0.794872,0.794872,0.807692,0.779221,0.779221,0.779221,0.802632,0.712329,0.712329,0.712329,0.802632,0.614286,0.528571,0.528571,0.528571,0.614286,0.565217,0.444444,0.444444,0.444444,0.565217,0.113924,0.802632,0.802632,0.802632,0.807692,0.794872,0.794872,0.794872,0.807692,0.786667,0.786667,0.786667,0.789474,0.712329,0.712329,0.712329,0.789474,0.605634,0.521127,0.521127,0.521127,0.605634,0.563380,0.450704,0.450704,0.450704,0.563380,0.1125
2,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.629630,0.15,0.134932,0.000000,0.100000,0.087500,0.091892,0.744304,0.761364,0.793478,0.329545,0.807229,0.825581,0.892272,0.879121,0.944444,0.966292,0.773810,0.879121,0.886040,0.845238,0.764706,0.963415,0.760563,0.9500,0.562701,0.807229,0.128571,0.741176,0.285714,0.822222,0.771588,0.758621,0.247059,0.838710,0.800000,0.773810,0.833333,0.242915,0.224719,0.273810,0.261905,0.021077,0.045455,0.011628,0.046512,0.011236,0.011364,0.759059,0.671756,0.75,0.75,0.25,0.771429,0.789474,0.779412,0.865672,0.194805,0.453125,0.007611,0.763158,0.763158,0.763158,0.846154,0.820513,0.820513,0.820513,0.846154,0.831169,0.831169,0.831169,0.842105,0.794521,0.794521,0.794521,0.842105,0.642857,0.571429,0.571429,0.571429,0.642857,0.565217,0.347222,0.347222,0.347222,0.565217,0.037975,0.776316,0.776316,0.776316,0.846154,0.820513,0.820513,0.820513

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [38]:
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.9253995300697726

In [39]:
predictions_test = lm.predict(X_test)
r2 = r2_score(y_test, predictions_test)
mse = mean_squared_error(y_test, predictions_test)
mae = mean_absolute_error(y_test, predictions_test)
rmse = math.sqrt(mse)

In [40]:
print("r2 score is:", r2)
print("mse is:", mse)
print("mae is:",mae)
print("rmse is:",rmse)

r2 score is: 0.9254566251979788
mse is: 3.5911448216642663
mae is: 1.4648287168565433
rmse is: 1.8950316149511244


In [41]:
fifa_v = pd.read_csv('./fifa21_validate.csv')
fifa_v.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,Right,7,"Feb 8, 2018","Jun 30, 2021",€975K,€5K,€0,"Jun 30, 2021 On Loan",230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34,48+2,48+2,48+2,48+0,47+0,47+0,47+0,48+0,48+2,48+2,48+2,50+2,51+2,51+2,51+2,50+2,59+2,61+2,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,Right,5,"Jan 20, 2020",NaN,€1.2M,€3K,€2.2M,2020 ~ 2020,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4 ★,3★,High,Low,1 ★,74,67,62,74,24,57,12,64+2,64+2,64+2,68+0,68+0,68+0,68+0,68+0,68+2,68+2,68+2,67+2,61+2,61+2,61+2,67+2,49+2,47+2,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,Right,17,"Nov 28, 2018",NaN,€120K,€500,€249K,2018 ~ 2020,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2 ★,1★,Medium,Medium,1 ★,56,52,53,55,26,53,3,18+2,18+2,18+2,15+0,17+0,17+0,17+0,15+0,17+2,17+2,17+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,Right,23,"Jan 10, 2020",NaN,€160K,€500,€464K,2020 ~ 2022,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3 ★,2★,Medium,Medium,1 ★,58,34,47,52,53,59,6,46+2,46+2,46+2,47+0,46+0,46+0,46+0,47+0,47+2,47+2,47+2,49+2,49+2,49+2,49+2,49+2,53+2,54+2,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,Right,5,"Jul 1, 2019",NaN,€2.3M,€13K,€4.3M,2019 ~ 2023,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4 ★,2★,Medium,Medium,1 ★,67,62,68,70,69,71,45,63+2,63+2,63+2,66+0,66+0,66+0,66+0,66+0,68+2,68+2,68+2,67+2,70+2,70+2,70+2,67+2,70+2,72+2,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70


In [42]:
fifa_v.columns = [i.lower().replace(' ', '_') for i in fifa_v.columns]
fifa_v.drop(columns = ['position','team_&_contract','loan_date_end', 'contract', 'joined'], axis = 1, inplace = True)

In [43]:
minus(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,"6'2""",194lbs,Right,7,€975K,€5K,€0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,67,50,50,50,48,47,47,47,48,50,50,50,52,53,53,53,52,61,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,"5'8""",154lbs,Right,5,€1.2M,€3K,€2.2M,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4 ★,3★,High,Low,1 ★,74,67,62,74,24,57,12,64,64,64,68,68,68,68,68,68,68,68,67,61,61,61,67,49,47,47,47,49,45,38,38,38,45,17,68,66,66,66,68,68,68,68,68,70,70,70,69,63,63,63,69,51,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,"6'2""",176lbs,Right,17,€120K,€500,€249K,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2 ★,1★,Medium,Medium,1 ★,56,52,53,55,26,53,3,18,18,18,15,17,17,17,15,17,17,17,16,18,18,18,16,16,18,18,18,16,16,18,18,18,16,53,54,20,20,20,15,17,17,17,15,19,19,19,18,20,20,20,18,18,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,"6'3""",170lbs,Right,23,€160K,€500,€464K,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3 ★,2★,Medium,Medium,1 ★,58,34,47,52,53,59,6,46,46,46,47,46,46,46,47,47,47,47,49,49,49,49,49,53,54,54,54,53,53,54,54,54,53,11,55,48,48,48,47,46,46,46,47,49,49,49,51,51,51,51,51,55,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,"6'2""",170lbs,Right,5,€2.3M,€13K,€4.3M,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4 ★,2★,Medium,Medium,1 ★,67,62,68,70,69,71,45,63,63,63,66,66,66,66,66,68,68,68,67,70,70,70,67,70,72,72,72,70,69,68,68,68,69,18,70,65,65,65,66,66,66,66,66,70,70,70,69,72,72,72,69,72,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,"5'8""",143lbs,Left,11,€325K,€1K,€731K,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,3 ★,2★,High,Medium,1 ★,76,28,46,55,53,57,4,44,44,44,51,48,48,48,51,48,48,48,53,48,48,48,53,58,52

In [44]:
star(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,"6'2""",194lbs,Right,7,€975K,€5K,€0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2,2,Low,High,1,64,30,50,50,66,74,34,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,67,50,50,50,48,47,47,47,48,50,50,50,52,53,53,53,52,61,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,"5'8""",154lbs,Right,5,€1.2M,€3K,€2.2M,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4,3,High,Low,1,74,67,62,74,24,57,12,64,64,64,68,68,68,68,68,68,68,68,67,61,61,61,67,49,47,47,47,49,45,38,38,38,45,17,68,66,66,66,68,68,68,68,68,70,70,70,69,63,63,63,69,51,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,"6'2""",176lbs,Right,17,€120K,€500,€249K,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2,1,Medium,Medium,1,56,52,53,55,26,53,3,18,18,18,15,17,17,17,15,17,17,17,16,18,18,18,16,16,18,18,18,16,16,18,18,18,16,53,54,20,20,20,15,17,17,17,15,19,19,19,18,20,20,20,18,18,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,"6'3""",170lbs,Right,23,€160K,€500,€464K,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3,2,Medium,Medium,1,58,34,47,52,53,59,6,46,46,46,47,46,46,46,47,47,47,47,49,49,49,49,49,53,54,54,54,53,53,54,54,54,53,11,55,48,48,48,47,46,46,46,47,49,49,49,51,51,51,51,51,55,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,"6'2""",170lbs,Right,5,€2.3M,€13K,€4.3M,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4,2,Medium,Medium,1,67,62,68,70,69,71,45,63,63,63,66,66,66,66,66,68,68,68,67,70,70,70,67,70,72,72,72,70,69,68,68,68,69,18,70,65,65,65,66,66,66,66,66,70,70,70,69,72,72,72,69,72,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,"5'8""",143lbs,Left,11,€325K,€1K,€731K,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,3,2,High,Medium,1,76,28,46,55,53,57,4,44,44,44,51,48,48,48,51,48,48,48,53,48,48,48,53,58,52,52,52,58,58,54,54,54,58,15,60

In [45]:
money(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,"6'2""",194lbs,Right,7,975000.0,5000.0,0.0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2,2,Low,High,1,64,30,50,50,66,74,34.0,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,67,50,50,50,48,47,47,47,48,50,50,50,52,53,53,53,52,61,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,"5'8""",154lbs,Right,5,1200000.0,3000.0,2200000.0,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4,3,High,Low,1,74,67,62,74,24,57,12.0,64,64,64,68,68,68,68,68,68,68,68,67,61,61,61,67,49,47,47,47,49,45,38,38,38,45,17,68,66,66,66,68,68,68,68,68,70,70,70,69,63,63,63,69,51,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,"6'2""",176lbs,Right,17,120000.0,500.0,249000.0,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2,1,Medium,Medium,1,56,52,53,55,26,53,3.0,18,18,18,15,17,17,17,15,17,17,17,16,18,18,18,16,16,18,18,18,16,16,18,18,18,16,53,54,20,20,20,15,17,17,17,15,19,19,19,18,20,20,20,18,18,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,"6'3""",170lbs,Right,23,160000.0,500.0,464000.0,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3,2,Medium,Medium,1,58,34,47,52,53,59,6.0,46,46,46,47,46,46,46,47,47,47,47,49,49,49,49,49,53,54,54,54,53,53,54,54,54,53,11,55,48,48,48,47,46,46,46,47,49,49,49,51,51,51,51,51,55,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,"6'2""",170lbs,Right,5,2300000.0,13000.0,4300000.0,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4,2,Medium,Medium,1,67,62,68,70,69,71,45.0,63,63,63,66,66,66,66,66,68,68,68,67,70,70,70,67,70,72,72,72,70,69,68,68,68,69,18,70,65,65,65,66,66,66,66,66,70,70,70,69,72,72,72,69,72,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,"5'8""",143lbs,Left,11,325000.0,1000.0,731000.0,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,3,2,High,Medium,1,76,28,46,55,53,57,4.0,44,44,44,51,48,48,4

In [46]:
weight(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,"6'2""",87.98,Right,7,975000.0,5000.0,0.0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2,2,Low,High,1,64,30,50,50,66,74,34.0,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,67,50,50,50,48,47,47,47,48,50,50,50,52,53,53,53,52,61,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,"5'8""",69.84,Right,5,1200000.0,3000.0,2200000.0,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4,3,High,Low,1,74,67,62,74,24,57,12.0,64,64,64,68,68,68,68,68,68,68,68,67,61,61,61,67,49,47,47,47,49,45,38,38,38,45,17,68,66,66,66,68,68,68,68,68,70,70,70,69,63,63,63,69,51,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,"6'2""",79.82,Right,17,120000.0,500.0,249000.0,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2,1,Medium,Medium,1,56,52,53,55,26,53,3.0,18,18,18,15,17,17,17,15,17,17,17,16,18,18,18,16,16,18,18,18,16,16,18,18,18,16,53,54,20,20,20,15,17,17,17,15,19,19,19,18,20,20,20,18,18,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,"6'3""",77.10,Right,23,160000.0,500.0,464000.0,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3,2,Medium,Medium,1,58,34,47,52,53,59,6.0,46,46,46,47,46,46,46,47,47,47,47,49,49,49,49,49,53,54,54,54,53,53,54,54,54,53,11,55,48,48,48,47,46,46,46,47,49,49,49,51,51,51,51,51,55,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,"6'2""",77.10,Right,5,2300000.0,13000.0,4300000.0,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4,2,Medium,Medium,1,67,62,68,70,69,71,45.0,63,63,63,66,66,66,66,66,68,68,68,67,70,70,70,67,70,72,72,72,70,69,68,68,68,69,18,70,65,65,65,66,66,66,66,66,70,70,70,69,72,72,72,69,72,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,"5'8""",64.85,Left,11,325000.0,1000.0,731000.0,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,3,2,High,Medium,1,76,28,46,55,53,57,4.0,44,44,44,51,48,48,48,51,4

In [47]:
height(fifa_v)

,id,name,age,nationality,club,bp,height,weight,foot,growth,value,wage,release_clause,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova,ls_pot,st_pot,rs_pot,lw_pot,lf_pot,cf_pot,rf_pot,rw_pot,lam_pot,cam_pot,ram_pot,lm_pot,lcm_pot,cm_pot,rcm_pot,rm_pot,lwb_pot,ldm_pot,cdm_pot,rdm_pot,rwb_pot,lb_pot,lcb_pot,cb_pot,rcb_pot,rb_pot,gk_pot
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,187.96,87.98,Right,7,975000.0,5000.0,0.0,230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2,2,Low,High,1,64,30,50,50,66,74,34.0,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,67,50,50,50,48,47,47,47,48,50,50,50,52,53,53,53,52,61,63,63,63,61,63,69,69,69,63,18
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,172.72,69.84,Right,5,1200000.0,3000.0,2200000.0,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4,3,High,Low,1,74,67,62,74,24,57,12.0,64,64,64,68,68,68,68,68,68,68,68,67,61,61,61,67,49,47,47,47,49,45,38,38,38,45,17,68,66,66,66,68,68,68,68,68,70,70,70,69,63,63,63,69,51,49,49,49,51,47,40,40,40,47,19
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,187.96,79.82,Right,17,120000.0,500.0,249000.0,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2,1,Medium,Medium,1,56,52,53,55,26,53,3.0,18,18,18,15,17,17,17,15,17,17,17,16,18,18,18,16,16,18,18,18,16,16,18,18,18,16,53,54,20,20,20,15,17,17,17,15,19,19,19,18,20,20,20,18,18,20,20,20,18,18,20,20,20,18,55
3,255120,N. Kenneh,16,England,Leeds United,CDM,190.50,77.10,Right,23,160000.0,500.0,464000.0,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3,2,Medium,Medium,1,58,34,47,52,53,59,6.0,46,46,46,47,46,46,46,47,47,47,47,49,49,49,49,49,53,54,54,54,53,53,54,54,54,53,11,55,48,48,48,47,46,46,46,47,49,49,49,51,51,51,51,51,55,56,56,56,55,55,56,56,56,55,13
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,187.96,77.10,Right,5,2300000.0,13000.0,4300000.0,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4,2,Medium,Medium,1,67,62,68,70,69,71,45.0,63,63,63,66,66,66,66,66,68,68,68,67,70,70,70,67,70,72,72,72,70,69,68,68,68,69,18,70,65,65,65,66,66,66,66,66,70,70,70,69,72,72,72,69,72,74,74,74,72,71,70,70,70,71,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,239074,S. Aw,21,Senegal,Gil Vicente FC,LB,172.72,64.85,Left,11,325000.0,1000.0,731000.0,197,59,23,42,51,22.0,207,51,30.0,29,41,56,344,74,78,63.0,50,79.0,245,31,66.0,68,50,30,232,59,52.0,51.0,34.0,36,59.0,167,52,57,58.0,51,6,15,7,10,13,1443,315,3,2,High,Medium,1,76,28,46,55,53,57,4.0,44,44,44,51,48,48,48,51,48,48,4

In [48]:
fifa_v.dropna(inplace=True)
fifa_v.drop_duplicates(inplace = True)

In [49]:
fifa_v_cat = fifa.select_dtypes('object').drop(['name', 'nationality', 'club'], axis = 1)
fifa_v_num = fifa._get_numeric_data().drop(['id'], axis = 1)
X_num_v = fifa_v_num.drop(['ova'], axis = 1)
y_v = fifa_v_num['ova']

In [50]:
transformer_v = MinMaxScaler().fit(X_num_v)
X_num_n_v = transformer_v.transform(X_num_v)
X_num_n_v = pd.DataFrame(X_num_n_v, columns=X_num_v.columns)

In [51]:
fifa_cat_hot_v = fifa_v_cat[['bp', 'foot']]
fifa_cat_ord_v = fifa_v_cat[['a/w', 'd/w']]

In [52]:
hot_encoder_v = OneHotEncoder(drop='first').fit(fifa_cat_hot_v)
hot_encoded_v = hot_encoder_v.transform(fifa_cat_hot_v).toarray()
hot_cols_v = hot_encoder_v.get_feature_names(input_features=fifa_cat_hot_v.columns)
hot_encoded_v = pd.DataFrame(hot_encoded_v, columns = [i.lower() for i in hot_cols_v])

In [53]:
ord_encoder_v = OrdinalEncoder(categories = [['Low', 'Medium', 'High'],['Low', 'Medium', 'High']]).fit(fifa_cat_ord_v)

ord_encoded_v= pd.DataFrame((ord_encoder_v.transform(fifa_cat_ord_v)), 
                           columns = fifa_cat_ord_v.columns)

In [54]:
transformer_ord_v = MinMaxScaler().fit(ord_encoded_v)
ord_encoded_n_v = transformer_ord_v.transform(ord_encoded_v)
ord_encoded_n_v = pd.DataFrame(ord_encoded_n_v, columns=fifa_cat_ord_v.columns)

In [55]:
X_v = pd.concat([ord_encoded_n_v, hot_encoded_v, X_num_n_v],axis=1)

In [56]:
predictions_test_v = lm.predict(X_v)
r2_v = r2_score(y_v, predictions_test_v)
mse_v = mean_squared_error(y_v, predictions_test_v)
mae_v = mean_absolute_error(y_v, predictions_test_v)
rmse_v = math.sqrt(mse)

In [57]:
print("r2 score is:", r2_v)
print("mse is:", mse_v)
print("mae is:",mae_v)
print("rmse is:",rmse_v)

r2 score is: 0.9254112962789006
mse is: 3.4885504766841477
mae is: 1.448850948705379
rmse is: 1.8950316149511244


# Results

In [58]:
results_for_p = lm.predict(X_v)

fifa_v_results = fifa_v['ova']

Results = pd.concat([fifa_v_results,pd.Series(results_for_p)],axis=1).head()

Results.rename(columns = {'ova':'Actual', 0:'Prediction'}, inplace = True)

Results.head()

,Actual,Prediction
0,67.0,63.960251
1,68.0,76.697217
2,54.0,77.231298
3,55.0,61.436883
4,70.0,66.290963


# Bonus Round

### Combining functions with validating data:

In [59]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import functools
from datetime import date
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn import linear_model

In [60]:
fifaa = pd.read_csv('./fifa21_train.csv')

In [61]:
def clean_data(df):
    df.columns = [i.lower().replace(' ', '_') for i in df.columns]
    df.drop(columns = ['position','team_&_contract','loan_date_end', 'contract', 'joined'], axis = 1, inplace = True)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    return df

def minus(df):
    lst = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']
    for i in lst:
        df[[i, i + '_pot']] = df[i].str.split('+', expand=True)
        df[i + '_pot'] = df[i + '_pot'].astype('int') + df[i].astype('int')
        df[i] = df[i].astype('int')
    return df

def money(df):
    
    lst_mny = ['value', 'wage', 'release_clause', 'hits']
    
    for i in lst_mny:
            
        
        df[i] = df[i].replace({'€': '', 'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(float)
        
    return df

def star(df):
    
    lst_str = ['w/f', 'sm', 'ir']
    
    for i in lst_str:
            
        df[[i]] = df[[i]].replace('★', '', regex=True)
        df[[i]] = df[[i]].astype('int')
        
    return df

def height(df):
    lst_height =['height']
    for i in lst_height:
        df[i] = df[i].replace({'"':''}, regex=True)
        df[i] = df[i].apply(lambda x: 12*int(x.split("'")[0])+int(x.split("'")[1]))
        df[i] = df[i]*2.54
    return df

def weight(df):
    
    lst_weight = ['weight']
    for i in lst_weight:
        df[i] = df[i].replace({'lbs':''}, regex=True).astype(int)
        df[i] = round(df[i]/2.205, 2) 
    
    return df

def l_model(df):
    
    df.dropna(inplace = True)
    
    df_cat = df.select_dtypes('object').drop(['name', 'nationality', 'club'], axis = 1)
    df_num = df._get_numeric_data().drop(['id'], axis = 1)
    X_num = df_num.drop(['ova'], axis = 1)
    y = df_num['ova']
    
    transformer = MinMaxScaler().fit(X_num)
    X_num_n = transformer.transform(X_num)
    X_num_n = pd.DataFrame(X_num_n, columns=X_num.columns)
    X_num_n.head()
    
    df_cat_hot = df_cat[['bp', 'foot']]
    df_cat_ord = df_cat[['a/w', 'd/w']]
    
    hot_encoder = OneHotEncoder(drop='first').fit(df_cat_hot)
    hot_encoded = hot_encoder.transform(df_cat_hot).toarray()
    hot_cols = hot_encoder.get_feature_names(input_features=df_cat_hot.columns)
    hot_encoded = pd.DataFrame(hot_encoded, columns = [i.lower() for i in hot_cols])
    
    ord_encoder = OrdinalEncoder(categories = [['Low', 'Medium', 'High'],['Low', 'Medium', 'High']]).fit(df_cat_ord)

    ord_encoded = pd.DataFrame((ord_encoder.transform(df_cat_ord)), columns = df_cat_ord.columns)
    
    transformer_ord = MinMaxScaler().fit(ord_encoded)
    ord_encoded_n = transformer_ord.transform(ord_encoded)
    ord_encoded_n = pd.DataFrame(ord_encoded_n, columns=df_cat_ord.columns)
    
    X = pd.concat([ord_encoded_n, hot_encoded, X_num_n],axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    lm = linear_model.LinearRegression()
    lm.fit(X_train,y_train)
    
    predictions = lm.predict(X_train)
    r2_score(y_train, predictions)
    
    predictions_test = lm.predict(X_test)
    r2 = r2_score(y_test, predictions_test)
    mse = mean_squared_error(y_test, predictions_test)
    mae = mean_absolute_error(y_test, predictions_test)
    rmse = math.sqrt(mse)
    
    print("r2 score is:", r2)
    print("mse is:", mse)
    print("mae is:",mae)
    print("rmse is:",rmse)
    
    results_for_v = lm.predict(X)

    df_results = df['ova']

    Results = pd.concat([df_results,pd.Series(results_for_v)],axis=1)

    Results.rename(columns = {'ova':'Target', 0:'Prediction'}, inplace = True)

     
    
    return Results

In [62]:
# https://mathieularose.com/function-composition-in-python

def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [63]:
fifa_clean = compose(l_model,minus,money,star,height,weight,clean_data)

In [64]:
fifa_clean(fifaa)

r2 score is: 0.9254566251979788
mse is: 3.5911448216642663
mae is: 1.4648287168565433
rmse is: 1.8950316149511244


,Target,Prediction
0,64.0,63.960251
1,77.0,76.697217
2,80.0,77.231298
3,59.0,61.436883
4,65.0,66.290963
...,...,...
11695,65.0,NaN
11696,65.0,NaN
11697,70.0,NaN
11698,67.0,NaN
